<a href="https://colab.research.google.com/github/Taeho-Kim-0322/Coding-Test/blob/master/%EC%8B%A4%EC%8A%B5_1_%EC%9D%B8%EA%B3%B5%EC%8B%A0%EA%B2%BD%EB%A7%9D%EA%B8%B0%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

※ 기본 개념 정리
1.   activation : 다음 층으로 어떻게 값을 넘길지 결정하는 부분. 가장 많이 사용되는 relu, sigmoid
2.   loss : 한 번 신경망이 실행될 때마다 오차 값을 추적하는 함수
3.   optimizer : 오차를 어떻게 줄여 나갈지 정하는 함수. e.g. SGD,...





TF 2.0 Tutorial(DL Camp) - MNIST 입니다.

해당 문서는 TensorFlow 공식 Tutorial을 기반으로 수정하였습니다.



먼저 프로그램에 텐서플로 라이브러리를 임포트합니다:

※ 자사 표준 라이브러리 TF

In [ ]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential

데이터셋을 로드하여 준비합니다.

공식적으로 제공하는 데이터셋은 [해당 링크](https://www.tensorflow.org/api_docs/python/tf/keras/datasets) 에서 확인할 수 있습니다.

In [ ]:
# data load
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data() 

# 0~255의 픽셀 값을 0~1로 조정
x_train, x_test = x_train / 255.0, x_test / 255.0

# shape 조정
x_train = x_train.reshape(60000,28,28,1)
x_test = x_test.reshape(10000,28,28,1)

# 정답 label을 길이10 벡터화
y_train = tf.one_hot(y_train, depth=10)
y_test = tf.one_hot(y_test, depth=10)

# Training set으로부터 Validation set을 따로 분리.
x_train, x_val = x_train[:50000], x_train[50000:]
y_train, y_val = y_train[:50000], y_train[50000:]

Dataset의 shape와 실제 값은 아래와 같습니다. 이미지의 각각의 pixel 값이라고 생각하시면 됩니다.

In [ ]:
print("Train set input shape : \t", x_train.shape)
print("Validation set input shape : \t", x_val.shape)
print("Test set input shape : \t\t", x_test.shape)
print()
print("Train set output shape : \t", y_train.shape)
print("Validation set output shape : \t", y_val.shape)
print("Test set output shape : \t", y_test.shape)

Train input : (5만건, 28 x 28, 흑백이미지)

Validation input : (1만건, 28x28, 흑백이미지)

Test input : (1만건, 28x28, 흑백이미지)


Train output : (5만건, 길이 10의 열 벡터)
...

해당하는 Dataset의 shape와 이미지는 아래와 같습니다.

In [ ]:
import matplotlib.pyplot as plt

INDEX = 100

plt.imshow(x_train[INDEX].reshape(28,28), cmap='gray_r')

print(x_train[INDEX].shape) # 가로세로 28, 흑백
print(y_train[INDEX]) # 정답 열벡터

※ input은 동일한 shape로 넣어줘야 하므로, 실 생활 사례에서는 전처리가 필요하다



인공신경망 아키텍처를 구성합니다. (교재상의 모델 구현)

In [ ]:
model = Sequential([
                    Flatten(), # 입력 데이터를 납작하게 1차원으로 눌러주는 역할
                    Dense(15, activation='sigmoid'), # Dense : Layer 한 개 층을 더함, 뉴런의 갯수를 15개로 지정, 활성화함수를 sigmoid
                    Dense(10, activation='softmax') # softmax는 가중합된 결과를 총 합이 1로 rescaling하여 가장 높은 값으로 분류한다.
])

훈련에 필요한 옵티마이저(optimizer)와 손실 함수를 선택합니다.

현재는 MSE(Quadratic error) Loss 함수와 SGD Optimizer를 사용할 예정입니다.

성능 평가는 분류정확도(accuracy)를 기준으로 합니다

In [ ]:
# 학습의 환경설정
my_opt = tf.keras.optimizers.SGD() # 경사하강법 Stochastic Gradient Descent
my_loss = tf.keras.losses.MeanSquaredError() # 모델의 예측값 ↔ 실제값 비교 Loss함수 : MSE
model.compile(optimizer=my_opt, loss=my_loss, metrics=['acc']) # accuracy 분류정확도

이제 모델을 5 Epoch 동안 학습하면서, 테스트할 예정입니다.

학습이 진행되는 동안 수집된 측정 지표를 바탕으로 최종 결과를 출력합니다.

학습은 미니배치 32를 단위로 하여 진행합니다.

In [ ]:
EPOCHS = 5
BATCH = 32
history = model.fit(x=x_train, y=y_train, batch_size=BATCH, epochs=EPOCHS, validation_data=(x_val, y_val), verbose=1)
# verbose = 실시간으로 로그를 찍어주는 방식을 설정함 1 ▶ 학습진행사항 다 찍어줌 0 ▶ 학습만 진행함

test 데이터에 대해 최종 성능을 체크합니다

In [ ]:
model.evaluate(x_test, y_test, verbose=2)

학습과정 동안의 Train Accuracy, Loss를 그래프로 확인할 수 있습니다. 



In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(5,5))
plt.plot(range(1,EPOCHS+1), history.history['acc'], label="Train")
plt.plot(range(1,EPOCHS+1), history.history['val_acc'], label="Validation")

plt.title("Accuracy Graph")
plt.xticks(range(1,EPOCHS+1), range(1,EPOCHS+1))
plt.legend(loc=2)

plt.show()

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(range(1,EPOCHS+1), history.history['loss'], label="Train")
plt.plot(range(1,EPOCHS+1), history.history['val_loss'], label="Validation")

plt.title("Loss Graph")
plt.xticks(range(1,EPOCHS+1), range(1,EPOCHS+1))
plt.legend(loc=3)

plt.show()

아래에는 Model의 구조와 Parameter 수를 확인할 수 있습니다.

In [ ]:
model.summary()
# Parameter : 최적화 하기 위해 찾아야 할 변수(가중치, 화살표)

# 실습 MISSION #1



* 첫 번째 레이어 node 수 조정하기 : node 수 50 
* 두 번째 레이어를 추가 : node 수 40
* 세 번째 레이어를 추가 : node 수 30
* 50 epoch동안 돌려보고 테스트 정확도를 확인해봅니다.
* 하이퍼파라미터 조정하기 : Learning Rate 10.0 (개선의 정도 사람이 설정함)



(힌트:learning rate은 파라미터를 업데이트할 때 이용됩니다. 어디에 넣어야 할까요?)



In [ ]:
model_1 = Sequential([
                    Flatten(),
                    Dense(50, activation='sigmoid'),
                    Dense(40, activation='sigmoid'),
                    Dense(30, activation='sigmoid'),
                    Dense(10, activation='softmax')
])

# 학습의 환경설정
my_opt = tf.keras.optimizers.SGD(learning_rate=10.0)
my_loss = tf.keras.losses.MeanSquaredError()
model_1.compile(optimizer=my_opt, loss=my_loss, metrics=['acc'])

In [ ]:
EPOCHS = 50
BATCH = 32
history_1 = model_1.fit(x=x_train, y=y_train, batch_size=BATCH, epochs=EPOCHS, validation_data=(x_val, y_val), verbose=1)

In [ ]:
model_1.evaluate(x_test, y_test, verbose=2)

In [ ]:
model_1.summary()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(5,5))
plt.plot(range(1,EPOCHS+1), history_1.history['acc'], label="Train")
plt.plot(range(1,EPOCHS+1), history_1.history['val_acc'], label="Validation")

plt.title("Accuracy Graph")
plt.xticks(range(1,EPOCHS+1), range(1,EPOCHS+1))
plt.legend(loc=2)

plt.show()

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(range(1,EPOCHS+1), history_1.history['loss'], label="Train")
plt.plot(range(1,EPOCHS+1), history_1.history['val_loss'], label="Validation")

plt.title("Loss Graph")
plt.xticks(range(1,EPOCHS+1), range(1,EPOCHS+1))
plt.legend(loc=3)

plt.show()

# 실습 MISSION #2

더 나은 성능을 내기 위해 네트워크를 수정해보자!

아래 항목을 자유로이 변경하고 **test set**에 대한 성능을 확인해봅시다

---



* 레이어 수 
* 레이어 당 노드 수
* learning rate
* epoch 수
* mini-batch size


In [ ]:
model_2 = Sequential([
                    Flatten(),
                    Dense(10, activation='sigmoid'),
                    Dense(10, activation='sigmoid'),
                    Dense(10, activation='sigmoid'),
                    Dense(10, activation='sigmoid'),
                    Dense(10, activation='sigmoid'),
                    Dense(10, activation='sigmoid'),
                    Dense(10, activation='softmax')
])

# 학습의 환경설정
my_opt = tf.keras.optimizers.SGD(learning_rate=10.0) # 경사하강법 Stochastic Gradient Descent
my_loss = tf.keras.losses.MeanSquaredError() # 모델의 예측값 ↔ 실제값 비교 Loss함수 : MSE
model_2.compile(optimizer=my_opt, loss=my_loss, metrics=['acc']) # accuracy 분류정확도

EPOCHS = 5
BATCH = 32
history_2 = model_2.fit(x=x_train, y=y_train, batch_size=BATCH, epochs=EPOCHS, validation_data=(x_val, y_val), verbose=1)

In [ ]:
model_2.evaluate(x_test, y_test, verbose=2)

In [ ]:
model_3 = Sequential([
                    Flatten(),
                    Dense(20, activation='sigmoid'),
                    Dense(20, activation='sigmoid'),
                    Dense(20, activation='sigmoid'),
                    Dense(10, activation='softmax')
])

# 학습의 환경설정
my_opt = tf.keras.optimizers.SGD(learning_rate=10.0) # 경사하강법 Stochastic Gradient Descent
my_loss = tf.keras.losses.MeanSquaredError() # 모델의 예측값 ↔ 실제값 비교 Loss함수 : MSE
model_3.compile(optimizer=my_opt, loss=my_loss, metrics=['acc']) # accuracy 분류정확도

EPOCHS = 5
BATCH = 32
history_3 = model_3.fit(x=x_train, y=y_train, batch_size=BATCH, epochs=EPOCHS, validation_data=(x_val, y_val), verbose=1)

In [ ]:
model_3.evaluate(x_test, y_test, verbose=2)

In [ ]:
model_4 = Sequential([
                    Flatten(),
                    Dense(30, activation='sigmoid'),
                    Dense(30, activation='sigmoid'),
                    Dense(10, activation='softmax')
])

# 학습의 환경설정
my_opt = tf.keras.optimizers.SGD(learning_rate=10.0) # 경사하강법 Stochastic Gradient Descent
my_loss = tf.keras.losses.MeanSquaredError() # 모델의 예측값 ↔ 실제값 비교 Loss함수 : MSE
model_4.compile(optimizer=my_opt, loss=my_loss, metrics=['acc']) # accuracy 분류정확도

EPOCHS = 5
BATCH = 32
history_4 = model_4.fit(x=x_train, y=y_train, batch_size=BATCH, epochs=EPOCHS, validation_data=(x_val, y_val), verbose=1)

In [ ]:
model_4.evaluate(x_test, y_test, verbose=2)

In [ ]:
 model_5 = Sequential([
                    Flatten(),
                    Dense(30, activation='sigmoid'),
                    Dense(30, activation='sigmoid'),
                    Dense(10, activation='softmax')
])
 
# 학습의 환경설정
my_opt = tf.keras.optimizers.SGD(learning_rate=10.0) # 경사하강법 Stochastic Gradient Descent
my_loss = tf.keras.losses.MeanSquaredError() # 모델의 예측값 ↔ 실제값 비교 Loss함수 : MSE
model_5.compile(optimizer=my_opt, loss=my_loss, metrics=['acc']) # accuracy 분류정확도

EPOCHS = 20
BATCH = 32
history_5 = model_5.fit(x=x_train, y=y_train, batch_size=BATCH, epochs=EPOCHS, validation_data=(x_val, y_val), verbose=1)

In [ ]:
model_5.evaluate(x_test, y_test, verbose=2)